In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Flatten,
    BatchNormalization,
    GlobalAveragePooling2D,
)
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
train_dir = "dataset/split/train/"
test_dir = "dataset/split/test/"
val_dir = "dataset/split/val/"

In [ ]:
batch_size = 128
learning_rate = 0.001
epochs = 50
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
dropout = 0.2

In [ ]:
def create_model():
    model = Sequential()

    pretrained_model = MobileNetV3Small(
        include_top=False, input_shape=(224, 224, 3), pooling="max", weights="imagenet"
    )

    pretrained_model.trainable = False

    pretrained_model = Model(
        inputs=pretrained_model.inputs, outputs=pretrained_model.layers[-2].output
    )
    model.add(pretrained_model)

    model.add(BatchNormalization())
    model.add(GlobalAveragePooling2D())

    model.add(Flatten())

    model.add(Dense(32, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(dropout))

    model.add(Dense(18, activation="softmax"))

    return model

In [ ]:
k=3
for i in range(3):
    train_data = image_dataset_from_directory(
        train_dir,
        labels='inferred',
        label_mode="categorical",
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    val_data = image_dataset_from_directory(
        val_dir,
        labels='inferred',
        label_mode="categorical",
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )

    test_data = image_dataset_from_directory(
        test_dir,
        labels='inferred',
        label_mode="categorical",
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )


    model=create_model()

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy',])

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs', callbacks=[tensorboard_callback])
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[tensorboard_callback])

    test_accuracy, test_loss=model.evaluate(test_data)
    print(f"Test Accuracy for Fold-{i+1}: {test_accuracy}")
